In [2]:
import json
import pandas as pd
from glob import glob

In [1]:
positions = [
    'pours',
    'contres',
    'abstentions'
]

In [3]:
def get_scrutin(filename):
    return json.load(open(filename))

scrutins = [ get_scrutin(f) for f in glob('data/**/VTAN*.json') ]

len(scrutins)

4417

In [5]:
def get_votes(scrutin):
    data = scrutin
    groupes = data['scrutin']['ventilationVotes']['organe']['groupes']['groupe']

    #print(scrutin)
    
    votes = [ (p, g['organeRef'], g['vote']['decompteNominatif'][p]['votant'], g['vote']['positionMajoritaire'])
           for g in groupes
           for p in positions
           if g['vote']['decompteNominatif'][p] != None
          ]

    df = (
        pd.concat([
            pd
            .DataFrame
            .from_records([v] if not isinstance(v, list) else v)
            .assign(
                position=p,
                organe=o,
                date=scrutin['scrutin']['dateScrutin'],
                scrutin=scrutin['scrutin']['uid'],
                groupe_majorite= gv
            )
            for (p, o, v, gv) in votes
        ])
        .assign(
            position = lambda df: df.position.replace({'pours': 'pour', 'contres': 'contre', 'abstentions': 'abstention'})
        )
    )

    return df

votes = pd.concat([ get_votes(scrutin) for scrutin in scrutins ])

votes

,acteurRef,mandatRef,parDelegation,position,organe,date,scrutin,groupe_majorite
0,PA609520,PM723508,false,pour,PO730964,2019-06-07,VTANR5L15V1934,pour
1,PA721296,PM723228,false,pour,PO730964,2019-06-07,VTANR5L15V1934,pour
2,PA718710,PM722432,false,pour,PO730964,2019-06-07,VTANR5L15V1934,pour
3,PA718756,PM722446,false,pour,PO730964,2019-06-07,VTANR5L15V1934,pour
4,PA718962,PM722514,false,pour,PO730964,2019-06-07,VTANR5L15V1934,pour
...,...,...,...,...,...,...,...,...
5,PA722284,PM723528,false,pour,PO730970,2017-10-21,VTANR5L15V180,pour
0,PA609332,PM723378,false,contre,PO730946,2017-10-21,VTANR5L15V180,contre
1,PA608172,PM723048,false,contre,PO730946,2017-10-21,VTANR5L15V180,contre
0,PA721062,PM723158,false,contre,PO730958,2017-10-21,VTANR5L15V180,contre


In [6]:
votes.to_csv('out/votes.csv', index=False)

In [7]:
votes.to_parquet('out/votes.par', index=False)

In [8]:
scrutins_df = pd.DataFrame.from_records([
    {
        'uid': s['scrutin']['uid'],
        'objet': s['scrutin']['titre'],
        'type': s['scrutin']['typeVote']['libelleTypeVote'],
        'date': s['scrutin']['dateScrutin']
    }
    for s in scrutins
])

scrutins_df

,uid,objet,type,date
0,VTANR5L15V1934,l'article 22 du projet de loi d'orientation de...,scrutin public ordinaire,2019-06-07
1,VTANR5L15V1924,l'amendement n° 386 de M. Gosselin et l'amende...,scrutin public ordinaire,2019-05-28
2,VTANR5L15V804,l'amendement n° 935 de M. Bazin à l'article 34...,scrutin public ordinaire,2018-06-03
3,VTANR5L15V1458,l'amendement n° 205 de Mme Obono après l'artic...,scrutin public ordinaire,2018-11-23
4,VTANR5L15V54,"la motion de rejet préalable, déposée par M. J...",scrutin public solennel,2017-07-24
...,...,...,...,...
4412,VTANR5L15V705,l'amendement n° 2449 de M. Garot à l'article 1...,scrutin public ordinaire,2018-05-28
4413,VTANR5L15V2339,l'amendement n° 64 de Mme Degois après l'artic...,scrutin public ordinaire,2019-12-04
4414,VTANR5L15V1173,l'amendement n° 2 de Mme Trastour-Isnart et le...,scrutin public ordinaire,2018-09-26
4415,VTANR5L15V3806,l'amendement n° 602 de M. Lachaud et les amend...,scrutin public ordinaire,2021-06-09


In [9]:
scrutins_df.type.value_counts()

scrutin public ordinaire    4288
scrutin public solennel      129
Name: type, dtype: int64

In [10]:
scrutins_df.to_csv('out/scrutins.csv', index=False)

In [11]:
def get_acteur(fn):
    data = json.load(open(fn))
    return {
        'uid': data['acteur']['uid']['#text'],
        'nom': data['acteur']['etatCivil']['ident']['nom'],
        'prenom': data['acteur']['etatCivil']['ident']['prenom']
    }

acteurs = pd.DataFrame.from_records([ get_acteur(fn) for fn in glob('data/json/**/PA*.json') ])

acteurs

,uid,nom,prenom
0,PA722170,Beaudouin-Hubiere,Sophie
1,PA719874,Jacquier-Laforge,Élodie
2,PA722134,Fiévet,Jean-Marie
3,PA719960,Faure-Muntian,Valéria
4,PA569,Bonnecarrère,Philippe
...,...,...,...
1237,PA722046,Bergé,Aurore
1238,PA643103,Ferrara,Jean-Jacques
1239,PA720928,Corceiro,David
1240,PA2371,Perol-Dumont,Marie-Françoise


In [12]:
acteurs.to_csv('out/acteurs.csv', index=False)

In [13]:
def get_organe(fn):
    data = json.load(open(fn))['organe']
    return data

organes = pd.DataFrame.from_records([ get_organe(fn) for fn in glob('data/json/**/PO*.json') ])

organes

,@xmlns,@xmlns:xsi,@xsi:type,uid,codeType,libelle,libelleEdition,libelleAbrege,libelleAbrev,viMoDe,...,regime,legislature,regimeJuridique,siteInternet,nombreReunionsAnnuelles,secretariat,listePays,positionPolitique,preseance,couleurAssociee
0,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,OrganeExtraParlementaire_type,PO528198,ORGEXTPARL,Conseil supérieur des gens de mer,du Conseil supérieur des gens de mer,Gens de mer,291,"{'dateDebut': '2011-12-30', 'dateAgrement': No...",...,5ème République,None,Article L. 5581-1 du code des transports,None,3,NaN,NaN,NaN,NaN,NaN
1,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,OrganeParlementaire_Type,PO784067,CMP,Commission mixte paritaire chargée de proposer...,de la commission mixte paritaire chargée de pr...,Climat et résilience,3875,"{'dateDebut': '2021-06-30', 'dateAgrement': No...",...,5ème République,15,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,NaN,NaN,NaN
2,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,OrganeParlementaire_Type,PO749043,MISINFOCOM,Mission d'information commune sur le foncier a...,de la mission d'information commune sur le fon...,Foncier agricole,MICAGRI,"{'dateDebut': '2018-02-07', 'dateAgrement': No...",...,5ème République,15,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,NaN,NaN,NaN
3,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,OrganeParlementaire_Type,PO772430,CMP,Commission mixte paritaire chargée de proposer...,Commission mixte paritaire chargée de proposer...,Second tour des municipales,3021,"{'dateDebut': '2020-06-11', 'dateAgrement': No...",...,5ème République,15,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,NaN,NaN,NaN
4,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,OrganeExtraParlementaire_type,PO418452,ORGEXTPARL,Conseil supérieur des archives,du Conseil supérieur des archives,Archives,246,"{'dateDebut': '2008-07-15', 'dateAgrement': No...",...,5ème République,None,Article L. 211-2-1 du code du patrimoine,https://francearchives.fr/fr/,2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1239,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,OrganeParlementaire_Type,PO768933,CMP,Commission mixte paritaire chargée de proposer...,de la commission mixte paritaire chargée de pr...,Lutte contre le gaspillage et économie circulaire,2274,"{'dateDebut': '2019-12-23', 'dateAgrement': No...",...,5ème République,15,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,NaN,NaN,NaN
1240,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,OrganeExterne_Type,PO774113,MINISTERE,Secrétariat d'État auprès des ministres de l'é...,"auprès des ministres de l'économie, des financ...",Transition numérique et communications électro...,NUM,"{'dateDebut': '2020-07-27', 'dateAgrement': No...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1241,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,OrganeParlementaire_Type,PO733920,MISINFO,Mission d’application de la loi n° 2016-339 du...,de la mission d’application de la loi n° 2016-...,Lutte contre les incivilités dans les transports,INCIVILITE,"{'dateDebut': '2017-07-05', 'dateAgrement': No...",...,5ème République,15,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,NaN,NaN,NaN
1242,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,OrganeParlementaire_Type,PO746992,GE,"Inondations, risques naturels et calamités agr...","INONDATIONS, RISQUES NATURELS ET CALAMITÉS AGR...","INONDATIONS, RISQUES NATURELS ET CALAMITÉS AGR...",INONDATION,"{'dateDebut': None, 'dateAgrement': '2017-12-2...",...,5ème République,15,NaN,NaN,NaN,"{

In [14]:
organes.to_csv('out/organes.csv', index=False)